In [1]:
from datasets import load_dataset, load_from_disk, Dataset
import numpy as np
import pandas as pd
from transformers import RobertaTokenizer, RobertaModel
import torch
import pickle
from torch.utils.data import TensorDataset, DataLoader

In [2]:
data_base = 'data'
dataset_name = 'cnn_dailymail'

data_path = data_base+'/'+dataset_name
sentence_emb_path = data_path+'/sentence_embs'
top_sentence_emb_path = data_path+'/top_sentence_embs'

In [3]:
summary_size = 4

In [4]:
# dataset = load_from_disk(
#     'data/cnn_dailymail/sentences/test/test_rewards'
# )

In [4]:
# dataset = load_from_disk(
#     'data/cnn_dailymail/sentences/validation/val_rewards'
# )

In [ ]:
dataset = load_from_disk(
    'data/cnn_dailymail/sentences/train/train_rewards'
)

In [5]:
def get_kept_article_ids_chunkl(split, chunk_num):
    with open(
          top_sentence_emb_path+'/'+split+'/'+split+'_data_article_ids_part_'+str(chunk_num)+".pkl", 
          "rb"
        ) as input_f:
            keep_a_ids = pickle.load(input_f)
    return keep_a_ids

In [7]:
keep_a_ids = [get_kept_article_ids_chunkl('train',p) for p in [1,2,3,4,5]]

In [7]:
# keep_a_ids = [get_kept_article_ids_chunkl('validation',p) for p in [1]]

In [6]:
# keep_a_ids = [get_kept_article_ids_chunkl('test',p) for p in [1]]

In [13]:
# dataset

In [7]:
def get_future_rewards(step_rewards):
    return np.flip(np.cumsum(np.flip(step_rewards)))

In [8]:
# splits = ['train','test','validation']
# a_id_len = dict([
#     (s,dict()) for s in splits
# ])

# arts = []
# a_top_gold_actions = []
# a_top_gold_future_rewards = []
# for split in splits:
#     if split in ['validation','train']:
#         continue
#     else:
#         for (a_id, a_s_idx_list, a_gold_acts_list, a_gold_reward_list) in zip(
#             dataset['id'], dataset['article_top_sent'],
#             dataset['all_gold_actions'], dataset['all_gold_rewards']
#         ):
#             a_len = len(a_s_idx_list)
#             if a_len!=10:
#                 continue
#             arts.append(a_id)
#             a_gold_reward_list = np.array(a_gold_reward_list)
#             best_summary_index = np.argmax(
#                 a_gold_reward_list[
#                     np.arange(a_gold_reward_list.shape[0]),
#                     (a_gold_reward_list!=0).cumsum(1).argmax(1)
#                 ]
#             )
#             a_top_gold_actions.append(a_gold_acts_list[best_summary_index])
#             a_top_gold_future_rewards.append(get_future_rewards(a_gold_reward_list[best_summary_index]))            

In [10]:
# splits = ['train','test','validation']
# a_id_len = dict([
#     (s,dict()) for s in splits
# ])

# arts = []
# a_top_gold_actions = []
# a_top_gold_future_rewards = []
# for split in splits:
#     if split in ['test','train']:
#         continue
#     else:
#         for (a_id, a_s_idx_list, a_gold_acts_list, a_gold_reward_list) in zip(
#             dataset['id'], dataset['article_top_sent'],
#             dataset['all_gold_actions'], dataset['all_gold_rewards']
#         ):
#             a_len = len(a_s_idx_list)
#             if a_len!=10:
#                 continue
#             arts.append(a_id)
#             a_gold_reward_list = np.array(a_gold_reward_list)
#             best_summary_index = np.argmax(
#                 a_gold_reward_list[
#                     np.arange(a_gold_reward_list.shape[0]),
#                     (a_gold_reward_list!=0).cumsum(1).argmax(1)
#                 ]
#             )
#             a_top_gold_actions.append(a_gold_acts_list[best_summary_index])
#             a_top_gold_future_rewards.append(get_future_rewards(a_gold_reward_list[best_summary_index]))            

In [25]:
splits = ['train','test','validation']
a_id_len = dict([
    (s,dict()) for s in splits
])

arts = []
a_top_gold_actions = []
a_top_gold_future_rewards = []
for split in splits:
    if split in ['test','validation']:
        continue
    else:
        for (a_id, a_s_idx_list, a_gold_acts_list, a_gold_reward_list) in zip(
            dataset['article_id']['article_id'], dataset['top_sentences_index']['top_sentences_index'],
            dataset['all_gold_actions']['all_gold_actions'], dataset['all_gold_rewards']['all_gold_rewards']
        ):
            a_len = len(a_s_idx_list)
            if a_len!=10:
                continue
            arts.append(a_id)
            a_gold_reward_list = np.array(a_gold_reward_list)
            best_summary_index = np.argmax(
                a_gold_reward_list[
                    np.arange(a_gold_reward_list.shape[0]),
                    (a_gold_reward_list!=0).cumsum(1).argmax(1)
                ]
            )
            a_top_gold_actions.append(a_gold_acts_list[best_summary_index])
            a_top_gold_future_rewards.append(get_future_rewards(a_gold_reward_list[best_summary_index]))            

In [9]:
len(arts), sum([len(k) for k in keep_a_ids])

(11251, 11251)

In [10]:
# start = 0
# for i,a_ids in enumerate(keep_a_ids):
#     part = i+1
#     end = start+len(a_ids)
    
#     gold_action_chunk = a_top_gold_actions[start:end]
#     gold_rewards_chunk = a_top_gold_future_rewards[start:end]
#     with open(
#     top_sentence_emb_path+'/test/'+'test_gold_actions_part_'+str(part)+'.pkl', 
#         'wb') as pickle_file:
#         pickle.dump(gold_action_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(
#         top_sentence_emb_path+'/test/'+'test_gold_future_rewards_part_'+str(part)+'.pkl', 
#         'wb') as pickle_file:
#         pickle.dump(gold_rewards_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
#     print('Created chunk:',part,'- indices',start,'-',end)
#     start = end

Created chunk: 1 - indices 0 - 11251


In [12]:
# start = 0
# for i,a_ids in enumerate(keep_a_ids):
#     part = i+1
#     end = start+len(a_ids)
    
#     gold_action_chunk = a_top_gold_actions[start:end]
#     gold_rewards_chunk = a_top_gold_future_rewards[start:end]
#     with open(
#     top_sentence_emb_path+'/validation/'+'validation_gold_actions_part_'+str(part)+'.pkl', 
#         'wb') as pickle_file:
#         pickle.dump(gold_action_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
#     with open(
#         top_sentence_emb_path+'/validation/'+'validation_gold_future_rewards_part_'+str(part)+'.pkl', 
#         'wb') as pickle_file:
#         pickle.dump(gold_rewards_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
#     print('Created chunk:',part,'- indices',start,'-',end)
#     start = end

Created chunk: 1 - indices 0 - 13111


In [36]:
start = 0
for i,a_ids in enumerate(keep_a_ids):
    part = i+1
    end = start+len(a_ids)
    
    gold_action_chunk = a_top_gold_actions[start:end]
    gold_rewards_chunk = a_top_gold_future_rewards[start:end]
    with open(
    top_sentence_emb_path+'/train/'+'train_gold_actions_part_'+str(part)+'.pkl', 
        'wb') as pickle_file:
        pickle.dump(gold_action_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    with open(
        top_sentence_emb_path+'/train/'+'train_gold_future_rewards_part_'+str(part)+'.pkl', 
        'wb') as pickle_file:
        pickle.dump(gold_rewards_chunk, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
    print('Created chunk:',part,'- indices',start,'-',end)
    start = end

Created chunk: 1 - indices 0 - 60000
Created chunk: 2 - indices 60000 - 120000
Created chunk: 3 - indices 120000 - 180000
Created chunk: 4 - indices 180000 - 240000
Created chunk: 5 - indices 240000 - 282638
